# Creation of evaluation data
never really used in the experiments! 
There is no storing of any data in this notebook!

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
import torch
import torch.nn.functional as F

In [ ]:
# load the quality overview
quality = pd.read_csv("~/MasterThesis/quality.csv")

# More info about the sequences
sequences = pd.read_csv("~/MasterThesis/sequences.csv")

# filter based on the table above
useable = quality[
    #(quality["Vessel in Frame"] <3) &
    #(quality['LM configuration'] < 4) &
    #(quality['Vein contrast/cropping error'] < 3) &
    #(quality['Artery contrast'] < 3) &
    #(quality['Vein boundary'] < 3) &
    #(quality['Artery boundary'] < 3) &
    #(quality['Gain'] == 1) &
    #(quality['Artefacts'] < 3) &
    #(quality['Movement'] < 3) &
    (quality['Total'] < 20)
]
useable.rename(columns={'ID': 'Id'}, inplace=True)
useable

In [ ]:
train_ids = [124,
413,
419,
524,
545,
209,
216,
456,
420,
546,
214,
304,
316,
418,
523,
543,
544,
549,
601,
610,
626,
667,
668,
692,
752,
772,
773,
1046,
1048,
1092,
1122,
1123,
1145,
172,
176,
177,
 184,
 211,
 219,
 246,
 307,
 341,
 349,
 353,
 354,
 384,
 402,
 408,
 416,
 426,
 442,
 452,
 457,
 467,
 486,
 515,
 521,
 526,
 542,
 553,
 563,
 573,
 578,
 584,
 599,
 1055,
 1065,
 1072,
 1082,
 1091,
 1097,
 1098,
 1101,
 1111,
 1128,
 1131,
 1141,
 1146,
 1148,
 1149,
 1150,
 1156,
 1164,
 1166,
 1168,
 1169,
 1301,
 1311,
 1314,
 4,
 9,
 10,
 12,
 13,
 17,
 20,
 21,
 25,
 28,
 29,
 33,
 34,
 36,
 42,
 58,
 59,
 61,
 67,
 68,
 70,
 75,
 78,
 79,
 84,
 100,
 106,
 107,
 109,
 113,
 114,
 115,
 116,
 117,
 118,
 120,
 121,
 122,
 123,
 125,
 126,
 127,
 128,
 130,
 135,
 136,
 137,
 140,
 146,
 147,
 151,
 169,
 2709,
 2709,
 2710,
 2710,
 2713,
 2713,
 2714,
 2714,
 2733,
 2733,
 2754,
 2758,
 2761,
 2765,
 2765,
 2766,
 4774,
 4775,
 4778,
 4781,
 4796,
 4798,
 4799,
 4801,
 4804,
 4808,
 4810,
 4813,
 4814,
 4825,
 4834,
 4856,
 2185,
 2187,
 2199,
 1627,
 1643,
 1354,
 1352,
 1366,
 844,
 1367,
 1667,
 913,
 1668,
 917,
 918,
 1669,
 1671,
 1673,
 1676,
 931,
 934,
 1015,
 1723,
 1452,
 1464,
 1462,
 1526,
 1616,
 2065,
 2141,
 2143,
 977,
 1693,
 1715,
 1733,1735,1741,1497,1506,1519,1620,986,929,968,859,861,906,908,910,956,962,919,903,905,921,922,943,944,946,947,965,975,997,998,1778,2829,2830,2831,2732,2736,2841,2866,2867,2868,2874,2875,2764, 2885]

In [ ]:
test_IDs = useable[~useable.Id.isin(train_ids)].Id.to_numpy()
test_IDs

In [ ]:
path_to_data = "/share/data_ultraschall/compressions"

In [ ]:
ids_with_segs = []
for test_id in test_IDs:
    id_path = os.path.join(path_to_data, str(int(test_id)), "segmentations", "1")
    if os.path.isdir(id_path):
        if len(os.listdir(id_path)) != 0:
            ids_with_segs.append(test_id)

ids_with_segs

In [ ]:
selected_ids = np.random.choice(ids_with_segs,30, replace=False)
selected_ids = selected_ids.astype(np.int32)
selected_ids

In [ ]:
len(np.unique(selected_ids))

In [ ]:
# load the ids of available patients
available = pd.read_csv("/home/nicke/MasterThesis/available_US_probands.csv")
available = available.drop('Unnamed: 0', axis=1)
available = available[available['Anatomy'] != 'BACKGROUND']
available

In [ ]:
path_to_data = '/share/data_ultraschall/compressions'

In [ ]:
landmarks = pd.read_csv('/home/nicke/MasterThesis/landmarks.csv')
landmarks = landmarks[(landmarks['Start Frames'] != '[]') & (landmarks['End Frames'] != '[]')& (landmarks['End Frames'] != 'DNC')]
landmarks

In [ ]:
data = landmarks.merge(available, on='Id')
id_list = (data.Id.values).astype(str)

In [ ]:
def load_image_and_seg(path, image):
    
    # load image with PIL and normalize
    img = np.array(Image.open(os.path.join(path,"frames", image))) / 255
    #print(os.path.join(path,"frames", image))
    #print(os.path.join(path,"segmentations", "1", image))
    
    # load seg and normalize over the labels
    seg =np.array(Image.open(os.path.join(path,"segmentations", "1", image))) / 200
    
    return img, seg

def get_image_seg_pairs(prob_id):
    
    
    print(f"Working on id: {prob_id}")
    all_frames = sorted(os.listdir(os.path.join(path_to_data,prob_id,"frames")))

    first_available_frame = np.fromstring(landmarks[landmarks['Id']== int(prob_id)]['Start Frames'].iat[0].strip(']['), sep=',', dtype=int)
    last_available_frame = np.fromstring(landmarks[landmarks['Id']== int(prob_id)]['End Frames'].iat[0].strip(']['), sep=',', dtype=int)
    file_pairs = []
    for j,f_frame in enumerate(first_available_frame):
        i = 6
        while f_frame + i < last_available_frame[j]:
            file_pairs.append([all_frames[f_frame], all_frames[f_frame+i]])
            i = i+6
            if len(file_pairs) > 6:
                break
    
    frame_pairs = []
    seg_pairs = []
    
    # load the seg and frame for fixed and moving
    for fixed_file, moving_file in file_pairs:
        
        fixed, fixed_seg = load_image_and_seg(os.path.join(path_to_data, prob_id), fixed_file)
        moving, moving_seg = load_image_and_seg(os.path.join(path_to_data, prob_id), moving_file)
        
        if fixed_seg.max() == 0:
            continue
        if moving_seg.max() == 0:
            continue
        
        
        # and store them together
        frame_pairs.append([fixed,moving])
        seg_pairs.append([fixed_seg,moving_seg])
    
    return np.array(frame_pairs), np.array(seg_pairs)    
    

In [ ]:
frames = []
segs = []
ids = []
for prob_id in id_list:

    frame_pairs, seg_pairs = get_image_seg_pairs(prob_id)
    
    for pair in frame_pairs:
        frames.append(pair)
    for pair in seg_pairs:
        segs.append(pair)